**SELF BOOK 2**

In [1]:
import cv2 as cv
import numpy as np

**Assignment - 1**

In [ ]:
import cv2 as cv
import numpy as np
import glob
import time

def nothing(x):
    pass

def stitch_panorama(images):
    stitcher = cv.Stitcher_create()
    status, pano = stitcher.stitch(images)
    if status == cv.Stitcher_OK:
        return pano
    else:
        print('Stitching failed:', status)
        return None

def load_obj_wireframe(filename):
    vertices = []
    edges = set()
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('v '):
                parts = line.strip().split()
                vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
            elif line.startswith('f '):
                parts = line.strip().split()[1:]
                idxs = [int(p.split('/')[0])-1 for p in parts]
                for i in range(len(idxs)):
                    edge = tuple(sorted((idxs[i], idxs[(i+1)%len(idxs)])))
                    edges.add(edge)
    return np.array(vertices, dtype=np.float32), list(edges)

def calibrate_camera_from_snapshots(snapshots, chessboard_size=(9,6)):
    objp = np.zeros((chessboard_size[0]*chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    objpoints = []
    imgpoints = []
    for idx, img in enumerate(snapshots):
        if len(img.shape) == 3 and img.shape[2] == 3:
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        else:
            gray = img.copy()
        ret, corners = cv.findChessboardCorners(gray, chessboard_size, None)
        display_img = img.copy()
        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)
            cv.drawChessboardCorners(display_img, chessboard_size, corners, ret)
            msg = f'Chessboard detected ({idx+1})'
        else:
            msg = f'No chessboard ({idx+1})'
        cv.putText(display_img, msg, (30, 60), cv.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0) if ret else (0,0,255), 3)
        cv.imshow('Detection Result', display_img)
        cv.waitKey(800)
    cv.destroyWindow('Detection Result')
    if len(objpoints) > 0:
        img_shape = gray.shape[::-1]
        ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, img_shape, None, None)
        print('Camera matrix:\n', mtx)
        print('Distortion coefficients:\n', dist.ravel())
    else:
        print('No chessboard corners found in snapshots. Try again.')

# Load dinosaur wireframe from OBJ
dino_obj_path = 'assets/trex_model.obj'
dino_vertices, dino_edges = load_obj_wireframe(dino_obj_path)

cap = cv.VideoCapture(0)
grayMode = False
hsvMode = False
showHist = True
cannyMode = False
houghMode = False
snapshots = []
showTransform = True

cv.namedWindow('main_video')
cv.namedWindow('Transform Controls')
cv.createTrackbar('Tx', 'Transform Controls', 0, 200, nothing)
cv.createTrackbar('Ty', 'Transform Controls', 0, 200, nothing)
cv.createTrackbar('Rotation', 'Transform Controls', 0, 360, nothing)
cv.createTrackbar('Scale', 'Transform Controls', 50, 200, nothing)  # 50=1.0, 100=2.0, 25=0.5
cv.createTrackbar('Contrast', 'main_video', 50, 100, nothing)
cv.createTrackbar('Brightness', 'main_video', 50, 100, nothing)
cv.createTrackbar('Gaussian', 'main_video', 1, 20, nothing)
cv.createTrackbar('Bilateral_d', 'main_video', 5, 20, nothing)
cv.createTrackbar('Bilateral_sigmaColor', 'main_video', 50, 150, nothing)
cv.createTrackbar('Bilateral_sigmaSpace', 'main_video', 50, 150, nothing)
cv.createTrackbar('Canny1', 'main_video', 50, 255, nothing)
cv.createTrackbar('Canny2', 'main_video', 150, 255, nothing)
cv.createTrackbar('Hough_thresh', 'main_video', 100, 300, nothing)

print('Press S to take a snapshot. Press P to create panorama. Press K to calibrate camera. Other keys: G/H/E/L/C/X/Q')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if showTransform:
        tx = cv.getTrackbarPos('Tx', 'Transform Controls')
        ty = cv.getTrackbarPos('Ty', 'Transform Controls')
        rotation = cv.getTrackbarPos('Rotation', 'Transform Controls')
        scale = cv.getTrackbarPos('Scale', 'Transform Controls') / 50.0  # 1.0 at 50
    else:
        tx, ty, rotation, scale = 0, 0, 0, 1.0

    contrast = cv.getTrackbarPos('Contrast', 'main_video') / 50.0
    brightness = (cv.getTrackbarPos('Brightness', 'main_video') - 50) / 50.0
    ksize = cv.getTrackbarPos('Gaussian', 'main_video')
    if ksize % 2 == 0:
        ksize += 1
    if ksize < 1:
        ksize = 1
    bilateral_d = cv.getTrackbarPos('Bilateral_d', 'main_video')
    if bilateral_d < 1:
        bilateral_d = 1
    bilateral_sigmaColor = cv.getTrackbarPos('Bilateral_sigmaColor', 'main_video')
    bilateral_sigmaSpace = cv.getTrackbarPos('Bilateral_sigmaSpace', 'main_video')
    canny1 = cv.getTrackbarPos('Canny1', 'main_video')
    canny2 = cv.getTrackbarPos('Canny2', 'main_video')
    hough_thresh = cv.getTrackbarPos('Hough_thresh', 'main_video')

    # --- Apply translation ---
    M_translate = np.float32([[1, 0, tx], [0, 1, ty]])
    display_frame = cv.warpAffine(frame, M_translate, (frame.shape[1], frame.shape[0]))

    # --- Apply rotation ---
    center = (display_frame.shape[1]//2, display_frame.shape[0]//2)
    M_rotate = cv.getRotationMatrix2D(center, rotation, 1.0)
    display_frame = cv.warpAffine(display_frame, M_rotate, (display_frame.shape[1], display_frame.shape[0]))

    # --- Apply scaling ---
    display_frame = cv.resize(display_frame, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)

    # --- Color/Filter Modes ---
    if grayMode:
        display_frame = cv.cvtColor(display_frame, cv.COLOR_BGR2GRAY)
    elif hsvMode:
        hsv = cv.cvtColor(display_frame, cv.COLOR_BGR2HSV)
        display_frame = hsv[:, :, 0]

    display_frame = cv.convertScaleAbs(display_frame, alpha=contrast, beta=brightness * 100)

    if ksize > 1:
        display_frame = cv.GaussianBlur(display_frame, (ksize, ksize), 0)
    if bilateral_d > 1:
        display_frame = cv.bilateralFilter(display_frame, bilateral_d, bilateral_sigmaColor, bilateral_sigmaSpace)

    if cannyMode or houghMode:
        if len(display_frame.shape) == 3:
            canny_input = cv.cvtColor(display_frame, cv.COLOR_BGR2GRAY)
        else:
            canny_input = display_frame
        edges = cv.Canny(canny_input, canny1, canny2)
        if cannyMode and not houghMode:
            display_frame = edges
    if houghMode:
        color_frame = frame.copy()
        lines = cv.HoughLinesP(edges, 1, np.pi/180, hough_thresh, minLineLength=50, maxLineGap=10)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv.line(color_frame, (x1, y1), (x2, y2), (0,255,0), 2)
        display_frame = color_frame

    # --- Chessboard detection and dinosaur AR overlay ---
    chessboard_size = (9,6)
    objp = np.zeros((chessboard_size[0]*chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    ar_frame = display_frame.copy() if len(display_frame.shape) == 3 else cv.cvtColor(display_frame, cv.COLOR_GRAY2BGR)
    if len(ar_frame.shape) == 3 and ar_frame.shape[2] == 3:
        gray_ar = cv.cvtColor(ar_frame, cv.COLOR_BGR2GRAY)
    else:
        gray_ar = ar_frame.copy()
    ret_cb, corners = cv.findChessboardCorners(gray_ar, chessboard_size, None)
    if ret_cb:
        cv.drawChessboardCorners(ar_frame, chessboard_size, corners, ret_cb)
        # Camera matrix and distortion (dummy values for AR, real values after calibration)
        h, w = ar_frame.shape[:2]
        focal_length = w
        center = (w/2, h/2)
        camera_matrix = np.array([[focal_length, 0, center[0]], [0, focal_length, center[1]], [0, 0, 1]], dtype=np.float32)
        dist_coeffs = np.zeros((4,1))
        # SolvePnP
        ret_pnp, rvecs, tvecs = cv.solvePnP(objp, corners, camera_matrix, dist_coeffs)
        # Project dinosaur wireframe
        imgpts, _ = cv.projectPoints(dino_vertices, rvecs, tvecs, camera_matrix, dist_coeffs)
        imgpts = imgpts.reshape(-1,2).astype(int)
        for i,j in dino_edges:
            pt1 = tuple(imgpts[i])
            pt2 = tuple(imgpts[j])
            cv.line(ar_frame, pt1, pt2, (0,255,255), 1)
        cv.putText(ar_frame, 'Dinosaur AR', (30, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
    display_frame = ar_frame

    cv.imshow('main_video', display_frame)

    if showHist:
        if len(display_frame.shape) == 2:
            hist = cv.calcHist([display_frame], [0], None, [256], [0,256])
            hist_img = np.zeros((200, 256, 3), dtype=np.uint8)
            cv.normalize(hist, hist, 0, 200, cv.NORM_MINMAX)
            for x, y in enumerate(hist):
                cv.line(hist_img, (x, 200), (x, 200-int(y)), (255,255,255), 1)
        else:
            hist_img = np.zeros((200, 256, 3), dtype=np.uint8)
            colors = [(255,0,0), (0,255,0), (0,0,255)]
            for i, col in enumerate(colors):
                hist = cv.calcHist([display_frame], [i], None, [256], [0,256])
                cv.normalize(hist, hist, 0, 200, cv.NORM_MINMAX)
                for x, y in enumerate(hist):
                    cv.line(hist_img, (x, 200), (x, 200-int(y)), col, 1)
        cv.imshow('Histogram', hist_img)

    key = cv.waitKey(100) & 0xFF

    if key == ord('g'):
        grayMode = not grayMode
        hsvMode = False
        print("GREY MODE" if grayMode else "NORMAL MODE")
    elif key == ord('h'):
        hsvMode = not hsvMode
        grayMode = False
        print("HSV MODE" if hsvMode else "NORMAL MODE")
    elif key == ord('e'):
        cannyMode = not cannyMode
        print("CANNY MODE" if cannyMode else "NORMAL MODE")
    elif key == ord('l'):
        houghMode = not houghMode
        print("HOUGH MODE" if houghMode else "NORMAL MODE")
    elif key == ord('c'):
        showHist = not showHist
        if not showHist:
            cv.destroyWindow('Histogram')
    elif key == ord('x'):
        showTransform = not showTransform
        if not showTransform:
            cv.destroyWindow('Transform Controls')
        else:
            cv.namedWindow('Transform Controls')
            cv.createTrackbar('Tx', 'Transform Controls', tx, 200, nothing)
            cv.createTrackbar('Ty', 'Transform Controls', ty, 200, nothing)
            cv.createTrackbar('Rotation', 'Transform Controls', rotation, 360, nothing)
            cv.createTrackbar('Scale', 'Transform Controls', int(scale*50), 200, nothing)
    elif key == ord('s'):
        snapshots.append(frame.copy())
        print(f'Snapshot taken! Total: {len(snapshots)}')
    elif key == ord('p') and len(snapshots) >= 2:
        print('Creating panorama...')
        pano = stitch_panorama(snapshots)
        if pano is not None:
            cv.imshow('Panorama', pano)
            cv.waitKey(0)
            cv.destroyWindow('Panorama')
    elif key == ord('k'):
        print('Starting camera calibration: capturing snapshots...')
        calib_snaps = []
        cancel_calib = False
        for i in range(10):
            print(f'Prepare chessboard for capture {i+1}/10')
            for t in range(5, 0, -1):
                ret, snap = cap.read()
                if ret:
                    timer_img = snap.copy()
                    cv.putText(timer_img, f'Timer: {t}s', (30, 60), cv.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 3)
                    cv.putText(timer_img, 'Press ESC to cancel', (30, 120), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                    cv.imshow('Calibration Snapshot', timer_img)
                    k = cv.waitKey(1000)
                    if k == 27:
                        cancel_calib = True
                        break
            if cancel_calib:
                print('Calibration cancelled by user.')
                break
            ret, snap = cap.read()
            if ret:
                calib_snaps.append(snap.copy())
                print(f'Captured calibration image {i+1}/10')
                cv.imshow('Captured Image', snap)
                k = cv.waitKey(800)
                if k == 27:
                    print('Calibration cancelled by user.')
                    cancel_calib = True
                    break
        cv.destroyWindow('Calibration Snapshot')
        if len(calib_snaps) > 0:
            cv.destroyWindow('Captured Image')
        if not cancel_calib and len(calib_snaps) > 0:
            calibrate_camera_from_snapshots(calib_snaps)
    elif key == ord('q'):
        print("QUIT")
        break

cap.release()
cv.destroyAllWindows()

Press S to take a snapshot. Press P to create panorama. Press K to calibrate camera. Other keys: G/H/E/L/C/X/Q


C:\Users\user\AppData\Local\Temp\ipykernel_25164\1183963236.py:215: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv.line(hist_img, (x, 200), (x, 200-int(y)), col, 1)


HOUGH MODE
Starting camera calibration: capturing snapshots...
Prepare chessboard for capture 1/10
Starting camera calibration: capturing snapshots...
Prepare chessboard for capture 1/10
Captured calibration image 1/10
Captured calibration image 1/10
Prepare chessboard for capture 2/10
Prepare chessboard for capture 2/10
Captured calibration image 2/10
Captured calibration image 2/10
Prepare chessboard for capture 3/10
Prepare chessboard for capture 3/10
Captured calibration image 3/10
Captured calibration image 3/10
Prepare chessboard for capture 4/10
Prepare chessboard for capture 4/10
Captured calibration image 4/10
Captured calibration image 4/10
Prepare chessboard for capture 5/10
Prepare chessboard for capture 5/10
Captured calibration image 5/10
Captured calibration image 5/10
Prepare chessboard for capture 6/10
Prepare chessboard for capture 6/10
Captured calibration image 6/10
Captured calibration image 6/10
Prepare chessboard for capture 7/10
Prepare chessboard for capture 7/